<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 3. Multilayer Perceptron</font></h1>

<h1><font color="#113D68" size=4>5. Proyecto de clasificación multiclase</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Importar clases, funciones y conjunto de datos](#section1)
* [2. Codificar la variable de salida](#section2)
* [3. Definir la red neuronal](#section3)
* [4. Evaluar el modelo](#section4)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En esta lección vamos a desarrollar y evaluar modelos de redes neuronales para problemas de clasificación multiclase. Así, veremos
* Cómo preparar datos de clasificación multiclase para modelar con redes neuronales.
* Cómo evaluar modelos de redes neuronales de Keras con scikit-learn.

In [12]:
# Las redes neuronales también pueden resolver problemas con nuestros datos simples (ya sean de Regresión como de Clasificación) además de los complejos (imágenes,audio,etc)
import tensorflow as tf
# Eliminar warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Importar clases, funciones y conjunto de datos</font>

Primero instalamos scikeras

In [4]:
!pip install scikeras

Podemos comenzar importando todas las clases, funciones y dataset que necesitaremos en este tutorial. 

In [13]:
# Problema de Clasificación Multiclase con el Iris Flowers Dataset
import pandas as pd # ahora lo hacemos con pandas en vez de numpy
from keras.models import Sequential
from keras.layers import Dense
# La nueva versión keras hay que poner delante "tensorflow"
from scikeras.wrappers import KerasClassifier
# La nueva versión keras has sustituido to_categorical así:
from tensorflow.keras.utils import to_categorical 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

# Cargamos el set de datos
dataframe = pd.read_csv("Datasets/iris.csv",header = None) # no tiene nombre de columnas (le agregamos índices numéricos)
print(dataframe)
dataset = dataframe.values # array numpy
X = dataset [:,0:4].astype(float)
Y = dataset [:,4]

       0    1    2    3               4
0    5.1  3.5  1.4  0.2     Iris-setosa
1    4.9  3.0  1.4  0.2     Iris-setosa
2    4.7  3.2  1.3  0.2     Iris-setosa
3    4.6  3.1  1.5  0.2     Iris-setosa
4    5.0  3.6  1.4  0.2     Iris-setosa
..   ...  ...  ...  ...             ...
145  6.7  3.0  5.2  2.3  Iris-virginica
146  6.3  2.5  5.0  1.9  Iris-virginica
147  6.5  3.0  5.2  2.0  Iris-virginica
148  6.2  3.4  5.4  2.3  Iris-virginica
149  5.9  3.0  5.1  1.8  Iris-virginica

[150 rows x 5 columns]


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el dataset [Iris](https://archive.ics.uci.edu/ml/datasets/Iris)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Codificar la variable de salida</font>

Como tenemos un problema de clasificación multiclase recordemos que debemos de utilizar One-Hot Encoding para poder formatear la salida. 

Por ejemplo, en este problema los tres valores de clase:
```
    Iris-setosa
    Iris-versicolor
    Iris-virginica
```

Por lo que convertimos esta salida en una codificación binaría como:
```
    Iris-setosa, Iris-versicolor, Iris-virginica
        1,             0,              0
        0,             1,              0
        0,             0,              1
```

Por tanto, 
1. Codificando primero las cadenas de manera coherente en números enteros utilizando la clase `LabelEncoder` de scikit-learn. 
2. Luego, conviertimos el vector de números enteros en One-Hot Enconding usando la función de Keras `to_categorical()`.

In [16]:
# Convertimos nuestra clase con 3 posibles valores de salida en una matriz booleana con el One-Hot Encoding. Esto lo hacemos para pasar de tener la variable de salida como string a numérica (trabajo más facil de la neurona)

# Codificar valores de clase como números enteros
encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(Y)
print(encoded_y) # Vemos que lo que hizo por ahora es transformar los 3 tipos de salidas string de Y a los valores numéricos 0-1-2 (no les sirve a las redes)

# Convierto los números enteros en variables ficticias (One-Hot Encoder) 
dummy_y = to_categorical(encoded_y) # se hace siempre en problemas de clasificación multiclase
print(dummy_y) # Me crea 3 variables de salida (3 columnas), cada una con un valor 0-1 plausible de poseer si corresponde o no a esa clase (matriz booleana)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Definir la red neuronal</font>

A continuación se muestra el procedimiento a la hora de crear la función a trabajar:
1. Crea una red simple completamente conectada con una capa oculta de 8 neuronas. 
2. La capa oculta utiliza una función de activación ReLu. 
3. Debido a que utilizamos One-Hot Encoding, la capa de salida debe crear 3 valores de salida, uno para cada clase. 
4. El valor de salida con el valor más grande se tomará como la clase predicha por el modelo. La topología quedaría así:
```
    4 inputs -> [8 hidden nodes] -> 3 outputs
```
5. Tendremos una función de activación Softmax en la capa de salida (siempre para Clasificación Multiclase). 
6. Finalmente, la red utiliza Adam con una función de pérdida logarítmica (`categorical_crossentropy`). (función de pérdida para un problema de Clasificación Multiclase).

In [17]:
# Creamos una red simple con una capa oculta con 8 neuronas (recordar 4 características y 3 posibles salidas). Usamos Relu para la oculta 
# El valor de salida con el número más alto (de 0 a 1) se tomará como la clase predicha y la capa de salida usará SoftMax (para asegurar que los valores de salidas esten entre 0-1, usándose como probabilidades predichas)

# Definimos el modelo base y lo almacenamos en una función
def baseline_model():
    # creamos el modelo
    model = Sequential() # modelo secuencial
    model.add(Dense(8,input_dim=4,activation='relu')) # capa oculta (8 neuronas/salidas,recibe 4 valores de entrada por características y func act ReLu)
    model.add(Dense(3,activation='softmax')) # capa de salida (3 neuronas/salidas, una por cada clase a predecir y func act SoftMax)
    # compilamos el modelo
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

Ahora podemos crear nuestro `KerasClassifier` para usarlo en scikit-learn. 

Aquí, pasamos el número de épocas como 200 y el tamaño de batch como 5 para usar al entrenar el modelo. 

In [18]:
# Creamos nuestro contenedor (wrapped)
estimator = KerasClassifier(build_fn=baseline_model,epochs=200,batch_size=5,verbose=0) # usamos la función que creamos y cargamos las épocas y batch size

<a id="section4"></a>
# <font color="#004D7F" size=6>4. Evaluar el modelo</font>

Ahora podemos evaluar nuestro modelo _(estimator)_ en nuestro conjunto de datos (`X` e `dummy_y`). Utilizando un procedimiento de validación cruzada de 10 veces (`kfold`)

In [19]:
# Acordarse que los valores de salida ahora están almacenados en las variables 'dummy_y'
# Usamos validación cruzada al poseer pocos datos, devolviéndonos el resultado para cada iteración

kfold = KFold(n_splits=10,shuffle=True)
results = cross_val_score(estimator,X,dummy_y,cv=kfold) # para el cálculo de métricas le paso el estimador (modelo con parámetros de ejecución establecidos), las variables x y ydummys que creamos al ser un problema de clasificación mmulticlase y el método de remustreo (entrenamiento)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100,results.std()*100))

# Probar añadir más capas o neuronas, ver hiperparámetros

C:\ProgramData\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\ProgramData\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, p

Accuracy: 98.67% (2.67%)


<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>